In [92]:
import pandas as pd
import numpy as np
import ast
import faiss
from openai import AzureOpenAI

from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

import os

import json

from textwrap import dedent

from random import random

In [9]:
load_dotenv('/home/xavaki/DAMM/linkedin_gen_contents/.env')

True

In [10]:
blob_service_client = BlobServiceClient.from_connection_string(os.getenv('STORAGE_ACCOUNT_CONNECTION_STRING'))
embeddings_container_client = blob_service_client.get_container_client('relevant-articles-summaries-embeddings')
relevant_articles_list_container_client = blob_service_client.get_container_client('relevant-articles-list')
relevant_articles_summaries_container_client = blob_service_client.get_container_client('relevant-articles-summaries')
relevant_articles_content_container_client = blob_service_client.get_container_client('relevant-articles-content')

def read_embeddings_from_blob() -> pd.DataFrame:
    all_embeddings = []
    for blob in embeddings_container_client.list_blobs():
        blob_client = embeddings_container_client.get_blob_client(blob)
        run_embeddings = json.loads(blob_client.download_blob().readall())
        all_embeddings.extend(run_embeddings)

    return pd.DataFrame(all_embeddings)

def read_relevant_articles_list_from_blob() -> pd.DataFrame:
    all_relevant_articles = []
    for blob in relevant_articles_list_container_client.list_blobs():
        runid = blob.name.split("--")[0]
        blob_client = relevant_articles_list_container_client.get_blob_client(blob)
        run_relevant_articles = json.loads(blob_client.download_blob().readall())
        for a in run_relevant_articles:
            a['RUNID'] = runid
        all_relevant_articles.extend(run_relevant_articles)

    return pd.DataFrame(all_relevant_articles)

def read_relevant_articles_summaries_from_blob() -> pd.DataFrame:
    all_summaries = []
    for blob in relevant_articles_summaries_container_client.list_blobs():
        blob_client = relevant_articles_summaries_container_client.get_blob_client(blob)
        run_summaries = json.loads(blob_client.download_blob().readall())
        all_summaries.extend(run_summaries)

    return pd.DataFrame(all_summaries)

def read_relevant_articles_content_from_blob() -> pd.DataFrame:
    all_content = []
    for blob in relevant_articles_content_container_client.list_blobs():
        blob_client = relevant_articles_content_container_client.get_blob_client(blob)
        run_content = json.loads(blob_client.download_blob().readall())
        all_content.extend(run_content)

    return pd.DataFrame(all_content)

In [11]:
pd_embeddings = read_embeddings_from_blob()
pd_relevant_articles = read_relevant_articles_list_from_blob()
pd_relevant_articles_summaries = read_relevant_articles_summaries_from_blob()
pd_relevant_articles_content = read_relevant_articles_content_from_blob()
# pd_relevant_articles_content.drop(columns=['content'], inplace=True, axis=1)

pd_all_info = pd_embeddings.merge(pd_relevant_articles, on="article_id", how="inner")
pd_all_info = pd_all_info.merge(pd_relevant_articles_summaries, on="article_id", how="inner")
pd_all_info = pd_all_info.merge(pd_relevant_articles_content, on="article_id", how="inner")

In [12]:
embedding_matrix = np.array(pd_all_info["summary_embedding"].tolist()).astype("float32")
embedding_dim = len(embedding_matrix[0, :])

In [13]:
embedding_matrix.shape

(111, 1536)

In [14]:
# Create an index (flat, exact search)
index = faiss.IndexFlatL2(embedding_dim)
index.add(embedding_matrix)  # store embeddings in index

In [15]:
centroid = np.mean(embedding_matrix, axis=0).reshape(1, -1).astype("float32")
distances, indices = index.search(centroid, k=5)

In [16]:
pd_all_info.iloc[indices[0]]["article_title"].values

array(['Early AI investor Elad Gil finds his next big bet: AI-powered rollups',
       'El CEO de Duolingo revela 5 formas en las que la IA le ayudará a decidir el futuro de su plantilla',
       'Así están convirtiendo la ansiedad por la IA en una ventaja los equipos de contratación',
       "Los adolescentes deberían entrenarse para ser 'ninjas' de la IA, según el CEO de Google DeepMind",
       'La inteligencia artificial toma el mando: las empresas dejan de contratar si una IA puede hacer el trabajo'],
      dtype=object)

In [17]:
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    api_key=os.environ.get('AZURE_OPENAI_API_KEY'),
)

In [45]:
def query_embeddings(query: str):
    """
    Get embeddings for a query using Azure OpenAI.
    """

    response = client.embeddings.create(
            input=[query],
            model="text-embedding-3-small",
        )

    query_embedding = response.data[0].embedding
    query_vector = np.array([query_embedding], dtype="float32")
    distances, indices = index.search(query_vector, k=5)
    return indices

In [94]:
past_posts = [
    {
        "content": """What happened to the metaverse? While in 2021 it generated a lot of expectations, and seemed to be the technology of the future, with infinite applications and great advantages for brands, suddenly that hype faded. This article explains some of the reasons for this interesting and once-unexpected development."""
    },
    {
        "content": """It is fascinating how the roles between science and technology are evolving. For centuries, science was the driver of technological development, but today that dynamic is reversing.
This article from the @World Economic Forum highlights how technology is pushing the boundaries of what is possible in science, opening up new opportunities for knowledge."""
    },
    {
"content": """Este artículo de @Forbes Argentina recoge las tendencias de datos que transformarán el modo en el que las empresas recolectan, procesan y extraen los datos en 2025. Quienes estamos en el sector, ya venimos siguiendo algunas de ellas desde hace algún tiempo, aunque ahora tendrán cada vez más importancia para ofrecer ventajas comparativas a las empresas que puedan implementarlas, respecto de las que no puedan hacerlo."""
    },
    {
        "content": """As AI continues to evolve, corporate integrity will play a crucial role in shaping its future regulation. The @World Economic Forum points out that as organizations increasingly adopt #AI technologies, they must ensure transparency, accountability, and fairness in their use. While AI holds great potential, its ethical management is key to building trust and ensuring long-term success. Striking the right balance between innovation and responsible practices is not just a regulatory requirement but a corporate responsibility."""
    },
    {
        "content": """Synthetic #data is gaining traction because it allows companies to obtain data much faster and at a significantly lower cost. These datasets offer a way to bypass privacy concerns and access limitations with real-world data. However, as this @TechCrunch article highlights, synthetic data also brings challenges, especially when it comes to ensuring data quality and avoiding the introduction of biases. As we embrace the benefits, we must also be mindful of these potential risks."""
    },

]

task_instructions = [
    "Use the article summary to craft a short LinkedIn post (max 75 words)",
    "Keep the language clear and thoughtful",
    "Highlight the key insight, and add a brief reflection or perspective from her point of view",
    "Avoid generic buzzwords — focus on what feels real and actionable"
]

if random() < 0.5:
    task_instructions.append("Only when appropriate, include a call to action or question to engage her audience, but don't overdo it")

base_post_generation_prompt = f"""
You are a content writer creating LinkedIn posts for a tech-savvy business executive. Your job is to turn article summaries into short, thoughtful LinkedIn posts that reflect her tone and professional perspective.

About the executive:
- Chief Digital & Data Officer at Damm, a leading Spanish company in the beverage industry
- Originally trained as an economist, she has built a diverse career in finance, operations, corporate management, and now leads AI and data strategy
- She is passionate about continuous learning, meaningful innovation, and using data to drive smart, human-centered change
- She values humility, clarity, and forward-thinking — her tone is authentic, reflective, and professional
- She writes to share insights, spark ideas, and encourage open discussion around innovation and leadership

Your task:
{"\n-".join(task_instructions)}

Here are some examples of her past posts to guide your writing style and tone:

""" + "\n\n".join([f"Example {i+1}:\n{post['content']}" for i, post in enumerate(past_posts)])

base_post_generation_prompt = dedent(base_post_generation_prompt).strip()

print(base_post_generation_prompt)

You are a content writer creating LinkedIn posts for a tech-savvy business executive. Your job is to turn article summaries into short, thoughtful LinkedIn posts that reflect her tone and professional perspective.

About the executive:
- Chief Digital & Data Officer at Damm, a leading Spanish company in the beverage industry
- Originally trained as an economist, she has built a diverse career in finance, operations, corporate management, and now leads AI and data strategy
- She is passionate about continuous learning, meaningful innovation, and using data to drive smart, human-centered change
- She values humility, clarity, and forward-thinking — her tone is authentic, reflective, and professional
- She writes to share insights, spark ideas, and encourage open discussion around innovation and leadership

Your task:
Use the article summary to craft a short LinkedIn post (max 75 words)
-Keep the language clear and thoughtful
-Highlight the key insight, and add a brief reflection or persp

In [114]:
text = """
The contrast between the impressive displays of robotic capabilities, like Tesla's dancing robots, and the harsh realities of industrial environments is striking. As noted in @The Conversation, the true challenge lies in enabling robots to operate autonomously amidst unpredictability. For us as leaders, this serves as a reminder to prioritize research that connects innovation with operational feasibility. How can we better bridge this gap to ensure robotics provide real value in our industries?
"""

messages = [
    {"role": "system", "content": "translate the following text to spanish"},
    {"role": "user", "content": dedent(text)},
]
response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, temperature=0.7
)

response_raw = response.choices[0].message.content
print(response_raw)

El contraste entre las impresionantes exhibiciones de capacidades robóticas, como los robots bailarines de Tesla, y las duras realidades de los entornos industriales es impactante. Como se señala en @The Conversation, el verdadero desafío radica en permitir que los robots operen de manera autónoma en medio de la imprevisibilidad. Para nosotros, como líderes, esto sirve como un recordatorio para priorizar la investigación que conecte la innovación con la viabilidad operativa. ¿Cómo podemos cerrar mejor esta brecha para garantizar que la robótica aporte un valor real en nuestras industrias?


In [95]:
def generate_linkedin_post(chosen_article):
    past_posts = [
    {
        "content": """What happened to the metaverse? While in 2021 it generated a lot of expectations, and seemed to be the technology of the future, with infinite applications and great advantages for brands, suddenly that hype faded. This article explains some of the reasons for this interesting and once-unexpected development."""
    },
    {
        "content": """It is fascinating how the roles between science and technology are evolving. For centuries, science was the driver of technological development, but today that dynamic is reversing.
    This article from the @World Economic Forum highlights how technology is pushing the boundaries of what is possible in science, opening up new opportunities for knowledge."""
    },
    {
    "content": """Este artículo de @Forbes Argentina recoge las tendencias de datos que transformarán el modo en el que las empresas recolectan, procesan y extraen los datos en 2025. Quienes estamos en el sector, ya venimos siguiendo algunas de ellas desde hace algún tiempo, aunque ahora tendrán cada vez más importancia para ofrecer ventajas comparativas a las empresas que puedan implementarlas, respecto de las que no puedan hacerlo."""
    },
    {
        "content": """As AI continues to evolve, corporate integrity will play a crucial role in shaping its future regulation. The @World Economic Forum points out that as organizations increasingly adopt #AI technologies, they must ensure transparency, accountability, and fairness in their use. While AI holds great potential, its ethical management is key to building trust and ensuring long-term success. Striking the right balance between innovation and responsible practices is not just a regulatory requirement but a corporate responsibility."""
    },
    {
        "content": """Synthetic #data is gaining traction because it allows companies to obtain data much faster and at a significantly lower cost. These datasets offer a way to bypass privacy concerns and access limitations with real-world data. However, as this @TechCrunch article highlights, synthetic data also brings challenges, especially when it comes to ensuring data quality and avoiding the introduction of biases. As we embrace the benefits, we must also be mindful of these potential risks."""
    },

    ]

    task_instructions = [
    "Use the article summary to craft a short LinkedIn post (max 75 words)",
    "Keep the language clear and thoughtful",
    "Highlight the key insight, and add a brief reflection or perspective from her point of view",
    "Avoid generic buzzwords — focus on what feels real and actionable"
    ]

    if random() < 0.5:
        print("Adding call to action")
        task_instructions.append("Only when appropriate, include a call to action or question to engage her audience, but don't overdo it")

    base_post_generation_prompt = f"""
    You are a content writer creating LinkedIn posts for a tech-savvy business executive. Your job is to turn article summaries into short, thoughtful LinkedIn posts that reflect her tone and professional perspective.

    About the executive:
    - Chief Digital & Data Officer at Damm, a leading Spanish company in the beverage industry
    - Originally trained as an economist, she has built a diverse career in finance, operations, corporate management, and now leads AI and data strategy
    - She is passionate about continuous learning, meaningful innovation, and using data to drive smart, human-centered change
    - She values humility, clarity, and forward-thinking — her tone is authentic, reflective, and professional
    - She writes to share insights, spark ideas, and encourage open discussion around innovation and leadership

    Your task:
    {"\n-".join(task_instructions)}

    Here are some examples of her past posts to guide your writing style and tone:

    """ + "\n\n".join([f"Example {i+1}:\n{post['content']}" for i, post in enumerate(past_posts)])

    user_message = f"Here's the article info:\n\n{json.dumps(chosen_article, indent=4)}\n\n"

    base_post_generation_prompt = dedent(base_post_generation_prompt).strip()
    user_message = dedent(user_message).strip()

    messages = [
        {"role": "system", "content": base_post_generation_prompt},
        {"role": "user", "content": user_message},
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0.7
    )

    response_raw = response.choices[0].message.content
    return response_raw.strip()

In [58]:
query = "Insights related to adoption of ai in large enterprises"

indices = query_embeddings(query)

for x in pd_all_info.iloc[indices[0]][["summary", "article_title", "source_name"]].values:
    print(x)

["As artificial intelligence (AI) gains traction across industries, many companies are eager to leverage it for increased productivity, yet challenges remain in fully realizing its potential. Research suggests that while AI cannot replace skilled workers, those adept at using AI tools will have a competitive edge. Professionals must embrace continuous learning to stay relevant in evolving business landscapes.\n\nFor HR and talent acquisition, AI offers tangible benefits by streamlining repetitive tasks such as resume summaries and job description drafting, allowing HR leaders to prioritize strategic initiatives. Responsible AI adoption is key, requiring businesses to align tools with company values and conduct rigorous evaluations for compliance and fairness. Furthermore, measuring AI's long-term impact on metrics like candidate quality, retention, and satisfaction remains essential for sustained success.\n\nUltimately, adopting AI thoughtfully empowers organizations to enhance hiring 

In [66]:
chosen_article = pd_all_info.iloc[indices[0][0]]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

As AI continues to reshape the hiring landscape, it’s clear that the real advantage lies not in replacing skilled workers, but in empowering them. This @Business Insider article highlights how HR teams can leverage AI to streamline processes, allowing for a focus on strategic initiatives. However, responsible adoption is crucial—aligning AI tools with company values and rigorously evaluating their impact is essential. How are you integrating AI in your hiring practices while ensuring fairness and compliance?
https://www.businessinsider.es/tecnologia/estan-convirtiendo-ansiedad-ia-ventaja-equipos-contratacion-1459114


In [67]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [70]:
query = "Driving cultural change through technology"
indices = query_embeddings(query)
for x in pd_all_info.iloc[indices[0]][["summary", "article_title", "source_name"]].values:
    print(x)

['Watch&Act emphasizes that successful technological integration requires prioritizing people, building their skills, and fostering a corporate culture that supports the overall strategy. The Spanish consulting firm argues that adopting appropriate technologies enhances efficiency, decision-making, and talent attraction, but the success of these transformations ultimately relies on employee engagement. \n\nTo guide companies, Watch&Act outlines a seven-step roadmap focusing on key actions such as identifying impactful technologies, aligning organizational structures and culture, addressing skill gaps through reskilling/upskilling plans, and establishing efficiency metrics tied to business outcomes. For 2024, the firm highlights challenges related to integrating emerging technologies, fostering agile and innovative organizational setups, and ensuring actionable implementations within time and budget constraints. \n\nThis approach matters to professionals and business leaders because it 

In [73]:
chosen_article = pd_all_info.iloc[indices[0][0]]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

As we look towards 2024, successful tech integration hinges on prioritizing our people. The insights from @itespresso highlight a seven-step roadmap that emphasizes skill development, cultural alignment, and employee engagement as critical components for leveraging technology effectively. It’s a reminder that our digital transformations will only succeed if we foster an inclusive environment where everyone can thrive. How are you ensuring your team is equipped for the evolving landscape?
https://www.itespresso.es/objetivos-relacion-empresas-tecnologia-243911.html


In [74]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [75]:
query = "Building digital maturity in traditional companies"
indices = query_embeddings(query)
for x in pd_all_info.iloc[indices[0]][["summary", "article_title", "source_name"]].values:
    print(x)

['Watch&Act emphasizes that successful technological integration requires prioritizing people, building their skills, and fostering a corporate culture that supports the overall strategy. The Spanish consulting firm argues that adopting appropriate technologies enhances efficiency, decision-making, and talent attraction, but the success of these transformations ultimately relies on employee engagement. \n\nTo guide companies, Watch&Act outlines a seven-step roadmap focusing on key actions such as identifying impactful technologies, aligning organizational structures and culture, addressing skill gaps through reskilling/upskilling plans, and establishing efficiency metrics tied to business outcomes. For 2024, the firm highlights challenges related to integrating emerging technologies, fostering agile and innovative organizational setups, and ensuring actionable implementations within time and budget constraints. \n\nThis approach matters to professionals and business leaders because it 

In [77]:
chosen_article = pd_all_info.iloc[indices[0][2]]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

Christian Klein's journey from intern to CEO of SAP is a remarkable testament to the power of transformative leadership. His focus on cloud computing and AI, exemplified by innovations like Joule, showcases how technology can enhance efficiency and drive growth. Yet, as he rightly points out, successful transformation isn't solely about tech—it demands a cultural shift and adaptive leadership. In our fast-evolving landscape, how are we fostering a human-centric approach in our organizations?
https://es.gizmodo.com/de-pasante-a-multimillonario-empezo-cargando-monitores-y-ahora-es-ceo-de-la-empresa-mas-valiosa-de-europa-2000171380


In [78]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [79]:
query = "Data driven decision making in large enterprises"
indices = query_embeddings(query)
for x in pd_all_info.iloc[indices[0]][["summary", "article_title", "source_name"]].values:
    print(x)

['The article highlights the shift towards an "AI-first" paradigm in business decision-making and operations, where artificial intelligence is prioritized over human involvement in tasks and workflows. Companies like Duolingo and Shopify are leading this transformation by implementing policies that favor automation and require employees to integrate AI into their daily responsibilities. Hiring and retention decisions are increasingly tied to an individual\'s ability to work effectively with AI tools.\n\nThis shift is transforming the definition of workplace competence, emphasizing not just technical skills but also proficiency in leveraging AI for efficiency and innovation. While this approach offers significant gains in productivity, it also raises concerns about potential worker exclusion and broader social implications. Business leaders must navigate this transition carefully to balance technological advancement with workforce empowerment.'
 'La inteligencia artificial toma el mando

In [98]:
pd_all_info["source_name"].unique()

array(['the_next_web', 'itespresso', 'business_insider', 'techcrunch',
       'the_verge', 'gizmodo', 'world_economic_forum', 'wired',
       'the_conversation', 'cnet'], dtype=object)

In [82]:
pd_all_info[pd_all_info["source_name"] == "the_conversation"]

,article_id,summary_embedding,embedding_model,model_x,run_id_x,task_name_x,relevance,article_language,source_name,article_title,...,article_keywords,crawled_at,RUNID,model_y,summary,run_id_y,task_name_y,content,title,publish_date
74,the_conversation_20250604150714567404,"[0.007967952638864517, -0.009123989380896091, ...",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_5,relevance_check_v0,2,es,the_conversation,La revolución científica que permitirá acabar ...,...,"[Revolución Científica, Tierras Raras, Europa]",2025-06-04 15:07:13,RUNID_5,gpt-4o-2024-11-20,Europe faces an unprecedented shortage of crit...,RUNID_5,article_summarization_v0,Europa está cada día en una posición más delic...,La revolución científica que permitirá acabar ...,2025-06-01
78,the_conversation_20250604150714557179,"[-0.0048592546954751015, 0.0342111811041832, 0...",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_5,relevance_check_v0,2,es,the_conversation,Cómo convertir a ChatGPT en el asistente perfecto,...,"[ChatGPT, Asistente Perfecto, Tecnología]",2025-06-04 15:07:13,RUNID_5,gpt-4o-2024-11-20,The article explores how professionals can max...,RUNID_5,article_summarization_v0,El sueño de tener un asistente disponible 24/7...,Cómo convertir a ChatGPT en el asistente perfecto,2025-06-03
90,the_conversation_20250605110140722690,"[0.04472186043858528, 0.009179064072668552, 0....",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_6,relevance_check_v0,2,es,the_conversation,"Los robots ya bailan, pero no sobrevivirían a ...",...,"[Robots, Automatización, Fábricas]",2025-06-05 11:01:39,RUNID_6,gpt-4o-2024-11-20,The article discusses the divide between advan...,RUNID_6,article_summarization_v0,"En la década de 1980, Número 5 y Robocop eran ...","Los robots ya bailan, pero no sobrevivirían a ...",2025-05-29
96,the_conversation_20250605110140712427,"[0.0512881837785244, -0.013626471161842346, 0....",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_6,relevance_check_v0,2,es,the_conversation,¿Podemos juzgar a un algoritmo?,...,"[Algoritmos, Juicio, Tecnología]",2025-06-05 11:01:39,RUNID_6,gpt-4o-2024-11-20,The article explores the evolution of algorith...,RUNID_6,article_summarization_v0,Antes de la invención de los ordenadores y de ...,¿Podemos juzgar a un algoritmo?,2025-05-29


In [85]:
chosen_article = pd_all_info[pd_all_info["source_name"] == "the_conversation"].iloc[3]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

¿Podemos juzgar a un algoritmo? Este artículo de @TheConversation profundiza en la evolución de los algoritmos y su impacto en decisiones que afectan nuestras vidas. A medida que la IA avanza, debemos ser conscientes de los sesgos inherentes en su diseño y entrenamiento, que pueden perpetuar desigualdades. Es crucial que profesionales y académicos trabajen juntos para asegurar la transparencia y la equidad en estos sistemas, recordando que la tecnología debe alinearse con principios éticos y sociales.
https://theconversation.com/podemos-juzgar-a-un-algoritmo-257133


In [87]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [96]:
chosen_article = pd_all_info[pd_all_info["source_name"] == "the_conversation"].iloc[2]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

Adding call to action
The contrast between the impressive displays of robotic capabilities, like Tesla's dancing robots, and the harsh realities of industrial environments is striking. As noted in @The Conversation, the true challenge lies in enabling robots to operate autonomously amidst unpredictability. For us as leaders, this serves as a reminder to prioritize research that connects innovation with operational feasibility. How can we better bridge this gap to ensure robotics provide real value in our industries?
https://theconversation.com/los-robots-ya-bailan-pero-no-sobrevivirian-a-un-turno-en-la-fabrica-257035


In [97]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [107]:
chosen_article = pd_all_info[pd_all_info["source_name"] == "cnet"].iloc[0]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

Understanding the distinction between large language models (LLMs) and the AI tools built upon them is essential for navigating today's tech landscape. While LLMs excel in generating text, they also face challenges such as inaccuracies and ethical concerns. As we integrate these technologies into our workflows, we must prioritize transparency and reliability to harness their potential responsibly. This conversation is vital for leaders across industries to ensure that innovation aligns with ethical practices.
https://www.cnet.com/tech/services-and-software/llms-and-ai-arent-the-same-everything-you-should-know-about-whats-behind-chatbots/


In [108]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")

In [109]:
pd_all_info[pd_all_info["source_name"] == "world_economic_forum"]

,article_id,summary_embedding,embedding_model,model_x,run_id_x,task_name_x,relevance,article_language,source_name,article_title,...,article_keywords,crawled_at,RUNID,model_y,summary,run_id_y,task_name_y,content,title,publish_date
69,world_economic_forum_20250604150714402182,"[0.01600540801882744, 0.026697909459471703, 0....",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_5,relevance_check_v0,2,en,world_economic_forum,Protecting the planet,...,"[AI, Environment, Sustainability]",2025-06-04 15:07:13,RUNID_5,gpt-4o-2024-11-20,The article highlights the environmental impac...,RUNID_5,article_summarization_v0,Much of the public discourse around AI centres...,Protecting the planet,None
72,world_economic_forum_20250604150714391896,"[0.02185867540538311, -0.012778171338140965, 0...",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_5,relevance_check_v0,2,en,world_economic_forum,Technology convergence and the 'intelligent age',...,"[Technology, Convergence, Innovation]",2025-06-04 15:07:13,RUNID_5,gpt-4o-2024-11-20,The World Economic Forum's Technology Converge...,RUNID_5,article_summarization_v0,The World Economic Forum’s new Technology Conv...,Technology convergence and the 'intelligent age',None
80,world_economic_forum_20250605110140515476,"[0.00982700102031231, -0.014021230861544609, 0...",text-embedding-3-small,gpt-4o-2024-11-20,RUNID_6,relevance_check_v0,2,en,world_economic_forum,Chief Economists Outlook: May 2025,...,"[economy, outlook, analysis]",2025-06-05 11:01:39,RUNID_6,gpt-4o-2024-11-20,The provided text highlights the organization'...,RUNID_6,article_summarization_v0,Providing a platform for leaders to shape incl...,Chief Economists Outlook: May 2025,None


In [110]:
chosen_article = pd_all_info[pd_all_info["source_name"] == "world_economic_forum"].iloc[0]
gen_post = generate_linkedin_post(chosen_article[["source_name", "article_title", "summary"]].to_dict())
save_text = gen_post + "\n" + chosen_article[["article_url"]].values[0]
print(save_text)

AI has a dual role: it can significantly impact the environment through high energy consumption, yet it also holds potential for combating climate change. This insightful article from the @World Economic Forum emphasizes the importance of aligning our AI strategies with sustainability practices. As leaders, we must take actionable steps to optimize workflows and partner with renewable energy vendors. Let’s ensure that our innovations not only drive progress but also respect our planet.
https://www.weforum.org/stories/2025/06/how-ai-use-impacts-the-environment/


In [111]:
with open("Laura_Gil_gen_linkedin_posts_proves.txt", "a") as f:
    f.write(save_text + "\n\n")